In [1]:
## This file implements neural networks and random forest on p0017presabs_qual.
## Due to the imbalanced data, we implement both over-sampling method and the combination of under- and over-sampling method.
## For fully-connected neural networks, the accuracy is 59.04% for over-sampling data, 65.06% after improvement.
## And it is 85.71% for combination data.
## To further improve the accuracy, we implement lasso regression to select important features, and thus have a new dataset.
## However, the new accuracy is 57.83% for over-sampling data and 65.06% after improvement.
## The new accuracy remains 85.71% for combination data.
## For random forest, the accuracy is 59.04% for over-sampling data and 85.71% for combination data.
## For random forest with cross-validation, the mean accuracy is 53.78% for over-sampling, and 93.46% for combination data.

In [1]:
from numpy.random import seed
seed(100)
import tensorflow
tensorflow.random.set_seed(123)

In [2]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_qual/p0017presabs_qual.csv')
df.shape

(255, 7004)

In [3]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [4]:
df['pheno']

0      2
1      0
2      2
3      2
4      2
5      2
6      2
7      0
8      1
9      1
10     1
11     2
12     2
13     1
14     0
15     0
16     2
17     2
18     0
19     0
20     1
21     0
22     0
23     2
24     0
25     1
26     0
27     0
28     1
29     0
      ..
225    1
226    2
227    2
228    0
229    2
230    1
231    1
232    2
233    2
234    0
235    1
236    0
237    0
238    2
239    0
240    0
241    0
242    0
243    2
244    2
245    0
246    0
247    2
248    0
249    1
250    2
251    1
252    0
253    2
254    2
Name: pheno, Length: 255, dtype: int64

In [5]:
df['pheno'].value_counts()

2    92
0    89
1    74
Name: pheno, dtype: int64

In [6]:
df.head()

,id,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTGATAATTT,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,...,group_315,group_381,group_426,group_4770,group_8913,group_9224,group_9493,group_9494,group_9905,pheno
0,107,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,109,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,120335,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2
4,120337,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,2


In [7]:
df_clean = df.drop(columns=['id'])

In [8]:
df_clean.shape

(255, 7003)

In [9]:
df_clean.head()

,TTTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAG,TTTTTTTGTAAATCTTTTCCATTTATTTCTTCGCTTTTCTCGCTACAATTTACAAATGGACTTGTTATCAACGTTAGGGCAATAAATGAAAGTAGTATAA,TTTTTTTGATAATTT,TTTTTTTCTTATTATACGAAATTTTCATATCATAGTAAAGTTTTTTACGAAAAAAACGTATTTAATGTTGACAATACGAAAATTTCGTATTATATTAGGT,TTTTTTTCGTAAAAAACTTTACTATGATATGAAAATTTCGTATAATAAGAAAAAAAGGAGGTAAGTAATATGAACAAAGAAAGAAATATTATTATAGCCA,TTTTTTTATTTTGCAATTTTTTATTTTCATTATAAACTTCCTTTCAAACACTGCTGAAATAGACGTCTTTTTCAAATAAGCATGATTAATACTTCAATTC,TTTTTTTATTTTACTCACCTCTAAAGCATAATTCCATATAAACTTGTTATTATCCGAAATTTTTGTATAATAGATTATTTTTGTTGGAGTAATTATTAGT,TTTTTTTATGTTATTATAATTAAAGTTTTCCATTGTTTTCCTCCTATAATAGCTTATCTGCAATCATCACAGCTAATAAATCGTTTTGTCTTATTGCTTC,TTTTTTTATGTTATAATCTTTCTAGACGTATTCAAAGGACGTCTTTTTAGATTGTATGTTATAGCTAGCCTTCCGGTTAATTTTTTGTTATGATGTGTTA,TTTTTTTATAACATTCAAAGTCTCACCATTGTCATTTGAATGATCATCAATAATAATTAATTCGTAATCAGTACTCTTCATTGTTTGATTTAATACAGAA,...,group_315,group_381,group_426,group_4770,group_8913,group_9224,group_9493,group_9494,group_9905,pheno
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2


In [10]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 7002) (255,)


In [11]:
# over-sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 92), (1, 92), (2, 92)]


Using TensorFlow backend.


In [12]:
# combination of under- and over- sampling
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

[(0, 21), (1, 16), (2, 30)]


In [13]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [14]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [15]:
############# Fully-Connected Neural Network ################

In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [17]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [18]:
model1_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_test_over, y_test_over))

Train on 193 samples, validate on 83 samples
Epoch 1/100
193/193 [==============================] - 0s 736us/step - loss: 1.2101 - accuracy: 0.4145 - val_loss: 1.6943 - val_accuracy: 0.4096
Epoch 2/100
193/193 [==============================] - 0s 259us/step - loss: 1.0555 - accuracy: 0.5389 - val_loss: 1.1141 - val_accuracy: 0.5663
Epoch 3/100
193/193 [==============================] - 0s 307us/step - loss: 1.0938 - accuracy: 0.6321 - val_loss: 1.0656 - val_accuracy: 0.5542
Epoch 4/100
193/193 [==============================] - 0s 269us/step - loss: 0.8767 - accuracy: 0.6114 - val_loss: 1.1532 - val_accuracy: 0.5181
Epoch 5/100
193/193 [==============================] - 0s 240us/step - loss: 0.7966 - accuracy: 0.6166 - val_loss: 1.1028 - val_accuracy: 0.5060
Epoch 6/100
193/193 [==============================] - 0s 252us/step - loss: 0.7506 - accuracy: 0.6736 - val_loss: 1.1065 - val_accuracy: 0.5301
Epoch 7/100
193/193 [==============================] - 0s 237us/step - loss: 0.7024 -

Epoch 57/100
193/193 [==============================] - 0s 251us/step - loss: 0.3326 - accuracy: 0.8705 - val_loss: 1.4252 - val_accuracy: 0.6145
Epoch 58/100
193/193 [==============================] - 0s 218us/step - loss: 0.3341 - accuracy: 0.8912 - val_loss: 1.4683 - val_accuracy: 0.6145
Epoch 59/100
193/193 [==============================] - 0s 197us/step - loss: 0.3339 - accuracy: 0.8964 - val_loss: 1.4959 - val_accuracy: 0.6024
Epoch 60/100
193/193 [==============================] - 0s 200us/step - loss: 0.3134 - accuracy: 0.9223 - val_loss: 1.4925 - val_accuracy: 0.5904
Epoch 61/100
193/193 [==============================] - 0s 208us/step - loss: 0.3110 - accuracy: 0.9067 - val_loss: 1.5068 - val_accuracy: 0.5783
Epoch 62/100
193/193 [==============================] - 0s 201us/step - loss: 0.3022 - accuracy: 0.9016 - val_loss: 1.4678 - val_accuracy: 0.5783
Epoch 63/100
193/193 [==============================] - 0s 211us/step - loss: 0.3121 - accuracy: 0.9275 - val_loss: 1.5133 -

In [20]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

83/83 [==============================] - 0s 245us/step
over-sampling test accuracy: 59.04%


In [29]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [30]:
model1_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [31]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 193 samples, validate on 83 samples
Epoch 1/1000
193/193 [==============================] - 0s 891us/step - loss: 1.1281 - accuracy: 0.3990 - val_loss: 0.9531 - val_accuracy: 0.5542
Epoch 2/1000
193/193 [==============================] - 0s 340us/step - loss: 1.0414 - accuracy: 0.5959 - val_loss: 0.9356 - val_accuracy: 0.5181
Epoch 3/1000
193/193 [==============================] - 0s 336us/step - loss: 0.9069 - accuracy: 0.6218 - val_loss: 0.9348 - val_accuracy: 0.5181
Epoch 4/1000
193/193 [==============================] - 0s 418us/step - loss: 0.8308 - accuracy: 0.6425 - val_loss: 0.9362 - val_accuracy: 0.5301
Epoch 5/1000
193/193 [==============================] - 0s 395us/step - loss: 0.7819 - accuracy: 0.6736 - val_loss: 0.9296 - val_accuracy: 0.6024
Epoch 6/1000
193/193 [==============================] - 0s 397us/step - loss: 0.7260 - accuracy: 0.7254 - val_loss: 0.9262 - val_accuracy: 0.5783
Epoch 7/1000
193/193 [==============================] - 0s 379us/step - loss: 0

193/193 [==============================] - 0s 376us/step - loss: 0.2096 - accuracy: 0.9275 - val_loss: 1.6618 - val_accuracy: 0.5542
Epoch 57/1000
193/193 [==============================] - 0s 319us/step - loss: 0.4351 - accuracy: 0.9067 - val_loss: 1.7209 - val_accuracy: 0.6145
Epoch 58/1000
193/193 [==============================] - 0s 288us/step - loss: 0.5402 - accuracy: 0.8756 - val_loss: 1.8029 - val_accuracy: 0.5301
Epoch 59/1000
193/193 [==============================] - 0s 315us/step - loss: 0.2607 - accuracy: 0.8860 - val_loss: 1.6213 - val_accuracy: 0.6024
Epoch 60/1000
193/193 [==============================] - 0s 297us/step - loss: 0.2878 - accuracy: 0.8912 - val_loss: 1.6153 - val_accuracy: 0.6265
Epoch 61/1000
193/193 [==============================] - 0s 288us/step - loss: 0.2629 - accuracy: 0.8860 - val_loss: 1.6992 - val_accuracy: 0.6024
Epoch 62/1000
193/193 [==============================] - 0s 290us/step - loss: 0.2381 - accuracy: 0.9326 - val_loss: 1.6263 - val_ac

Epoch 112/1000
193/193 [==============================] - 0s 312us/step - loss: 0.1451 - accuracy: 0.9430 - val_loss: 2.1589 - val_accuracy: 0.5904
Epoch 113/1000
193/193 [==============================] - 0s 297us/step - loss: 0.1277 - accuracy: 0.9585 - val_loss: 2.1718 - val_accuracy: 0.5904
Epoch 114/1000
193/193 [==============================] - 0s 288us/step - loss: 0.1222 - accuracy: 0.9534 - val_loss: 2.2004 - val_accuracy: 0.5783
Epoch 115/1000
193/193 [==============================] - 0s 298us/step - loss: 0.1279 - accuracy: 0.9482 - val_loss: 2.2339 - val_accuracy: 0.5904
Epoch 116/1000
193/193 [==============================] - 0s 314us/step - loss: 0.1250 - accuracy: 0.9585 - val_loss: 2.2549 - val_accuracy: 0.5904
Epoch 117/1000
193/193 [==============================] - 0s 300us/step - loss: 0.1239 - accuracy: 0.9534 - val_loss: 2.2337 - val_accuracy: 0.6024
Epoch 118/1000
193/193 [==============================] - 0s 285us/step - loss: 0.1202 - accuracy: 0.9534 - val_

193/193 [==============================] - 0s 311us/step - loss: 0.0940 - accuracy: 0.9637 - val_loss: 2.5620 - val_accuracy: 0.5904
Epoch 168/1000
193/193 [==============================] - 0s 335us/step - loss: 0.1618 - accuracy: 0.9482 - val_loss: 2.5141 - val_accuracy: 0.5783
Epoch 169/1000
193/193 [==============================] - 0s 297us/step - loss: 0.1315 - accuracy: 0.9534 - val_loss: 2.6534 - val_accuracy: 0.5904
Epoch 170/1000
193/193 [==============================] - 0s 297us/step - loss: 0.1644 - accuracy: 0.9378 - val_loss: 2.4173 - val_accuracy: 0.6145
Epoch 171/1000
193/193 [==============================] - 0s 294us/step - loss: 0.1220 - accuracy: 0.9534 - val_loss: 2.6240 - val_accuracy: 0.6024
Epoch 172/1000
193/193 [==============================] - 0s 299us/step - loss: 0.2298 - accuracy: 0.9119 - val_loss: 2.7402 - val_accuracy: 0.5783
Epoch 173/1000
193/193 [==============================] - 0s 298us/step - loss: 0.2219 - accuracy: 0.9119 - val_loss: 2.3997 - 

193/193 [==============================] - 0s 300us/step - loss: 0.2442 - accuracy: 0.9482 - val_loss: 2.9076 - val_accuracy: 0.5060
Epoch 223/1000
193/193 [==============================] - 0s 308us/step - loss: 0.1213 - accuracy: 0.9534 - val_loss: 2.6095 - val_accuracy: 0.6024
Epoch 224/1000
193/193 [==============================] - 0s 299us/step - loss: 0.1069 - accuracy: 0.9689 - val_loss: 2.6019 - val_accuracy: 0.6145
Epoch 225/1000
193/193 [==============================] - 0s 295us/step - loss: 0.2451 - accuracy: 0.9637 - val_loss: 2.7572 - val_accuracy: 0.6024
Epoch 226/1000
193/193 [==============================] - 0s 318us/step - loss: 0.8340 - accuracy: 0.8497 - val_loss: 2.4330 - val_accuracy: 0.5181
Epoch 227/1000
193/193 [==============================] - 0s 302us/step - loss: 0.4922 - accuracy: 0.8601 - val_loss: 2.5449 - val_accuracy: 0.5663
Epoch 228/1000
193/193 [==============================] - 0s 295us/step - loss: 0.3404 - accuracy: 0.9223 - val_loss: 2.2618 - 

...

193/193 [==============================] - 0s 293us/step - loss: 0.0563 - accuracy: 0.9689 - val_loss: 3.0131 - val_accuracy: 0.6506
Epoch 773/1000
193/193 [==============================] - 0s 321us/step - loss: 0.0533 - accuracy: 0.9741 - val_loss: 3.0300 - val_accuracy: 0.6506
Epoch 774/1000
193/193 [==============================] - 0s 291us/step - loss: 0.0520 - accuracy: 0.9741 - val_loss: 3.0293 - val_accuracy: 0.6506
Epoch 775/1000
193/193 [==============================] - 0s 310us/step - loss: 0.0509 - accuracy: 0.9741 - val_loss: 3.0297 - val_accuracy: 0.6506
Epoch 776/1000
193/193 [==============================] - 0s 305us/step - loss: 0.0546 - accuracy: 0.9689 - val_loss: 3.0168 - val_accuracy: 0.6506
Epoch 777/1000
193/193 [==============================] - 0s 298us/step - loss: 0.0537 - accuracy: 0.9741 - val_loss: 3.0908 - val_accuracy: 0.6506
Epoch 778/1000
193/193 [==============================] - 0s 320us/step - loss: 0.0541 - accuracy: 0.9689 - val_loss: 3.0737 - 

193/193 [==============================] - 0s 312us/step - loss: 0.0512 - accuracy: 0.9741 - val_loss: 3.3125 - val_accuracy: 0.6506
Epoch 828/1000
193/193 [==============================] - 0s 309us/step - loss: 0.0538 - accuracy: 0.9741 - val_loss: 3.3586 - val_accuracy: 0.6386
Epoch 829/1000
193/193 [==============================] - 0s 319us/step - loss: 0.0509 - accuracy: 0.9741 - val_loss: 3.3799 - val_accuracy: 0.6386
Epoch 830/1000
193/193 [==============================] - 0s 314us/step - loss: 0.0504 - accuracy: 0.9637 - val_loss: 3.3977 - val_accuracy: 0.6386
Epoch 831/1000
193/193 [==============================] - 0s 301us/step - loss: 0.0524 - accuracy: 0.9741 - val_loss: 3.4000 - val_accuracy: 0.6386
Epoch 832/1000
193/193 [==============================] - 0s 303us/step - loss: 0.0497 - accuracy: 0.9689 - val_loss: 3.4077 - val_accuracy: 0.6506
Epoch 833/1000
193/193 [==============================] - 0s 315us/step - loss: 0.0502 - accuracy: 0.9741 - val_loss: 3.4147 - 

193/193 [==============================] - 0s 314us/step - loss: 0.0540 - accuracy: 0.9689 - val_loss: 3.6623 - val_accuracy: 0.6506
Epoch 883/1000
193/193 [==============================] - 0s 320us/step - loss: 0.0440 - accuracy: 0.9741 - val_loss: 3.6735 - val_accuracy: 0.6386
Epoch 884/1000
193/193 [==============================] - 0s 307us/step - loss: 0.0502 - accuracy: 0.9741 - val_loss: 3.6681 - val_accuracy: 0.6386
Epoch 885/1000
193/193 [==============================] - 0s 311us/step - loss: 0.0441 - accuracy: 0.9689 - val_loss: 3.6778 - val_accuracy: 0.6386
Epoch 886/1000
193/193 [==============================] - 0s 298us/step - loss: 0.0476 - accuracy: 0.9741 - val_loss: 3.6775 - val_accuracy: 0.6386
Epoch 887/1000
193/193 [==============================] - 0s 298us/step - loss: 0.0522 - accuracy: 0.9689 - val_loss: 3.6747 - val_accuracy: 0.6386
Epoch 888/1000
193/193 [==============================] - 0s 325us/step - loss: 0.0509 - accuracy: 0.9689 - val_loss: 3.6608 - 

193/193 [==============================] - 0s 310us/step - loss: 0.0473 - accuracy: 0.9741 - val_loss: 3.7713 - val_accuracy: 0.6265
Epoch 938/1000
193/193 [==============================] - 0s 328us/step - loss: 0.0520 - accuracy: 0.9741 - val_loss: 3.7910 - val_accuracy: 0.6265
Epoch 939/1000
193/193 [==============================] - 0s 313us/step - loss: 0.0472 - accuracy: 0.9741 - val_loss: 3.8175 - val_accuracy: 0.6386
Epoch 940/1000
193/193 [==============================] - 0s 306us/step - loss: 0.0474 - accuracy: 0.9637 - val_loss: 3.8279 - val_accuracy: 0.6386
Epoch 941/1000
193/193 [==============================] - 0s 292us/step - loss: 0.0478 - accuracy: 0.9689 - val_loss: 3.8368 - val_accuracy: 0.6386
Epoch 942/1000
193/193 [==============================] - 0s 298us/step - loss: 0.0462 - accuracy: 0.9741 - val_loss: 3.8397 - val_accuracy: 0.6386
Epoch 943/1000
193/193 [==============================] - 0s 301us/step - loss: 0.0485 - accuracy: 0.9741 - val_loss: 3.8289 - 

193/193 [==============================] - 0s 317us/step - loss: 0.0568 - accuracy: 0.9741 - val_loss: 3.9226 - val_accuracy: 0.6506
Epoch 993/1000
193/193 [==============================] - 0s 321us/step - loss: 0.0503 - accuracy: 0.9741 - val_loss: 3.9479 - val_accuracy: 0.6506
Epoch 994/1000
193/193 [==============================] - 0s 306us/step - loss: 0.0487 - accuracy: 0.9741 - val_loss: 3.9152 - val_accuracy: 0.6506
Epoch 995/1000
193/193 [==============================] - 0s 315us/step - loss: 0.0510 - accuracy: 0.9689 - val_loss: 3.8772 - val_accuracy: 0.6506
Epoch 996/1000
193/193 [==============================] - 0s 313us/step - loss: 0.0487 - accuracy: 0.9741 - val_loss: 3.9446 - val_accuracy: 0.6024
Epoch 997/1000
193/193 [==============================] - 0s 298us/step - loss: 0.0562 - accuracy: 0.9637 - val_loss: 4.6830 - val_accuracy: 0.5904
Epoch 998/1000
193/193 [==============================] - 0s 316us/step - loss: 0.2059 - accuracy: 0.9534 - val_loss: 3.6941 - 

In [32]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

83/83 [==============================] - 0s 189us/step
over-sampling test accuracy: 65.06%


In [33]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [34]:
model1_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [35]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=100,
          validation_data=(X_test_comb, y_test_comb))

Train on 46 samples, validate on 21 samples
Epoch 1/100
46/46 [==============================] - 0s 3ms/step - loss: 1.9211 - accuracy: 0.3261 - val_loss: 0.5553 - val_accuracy: 0.7143
Epoch 2/100
46/46 [==============================] - 0s 465us/step - loss: 0.5648 - accuracy: 0.6522 - val_loss: 0.4913 - val_accuracy: 0.7619
Epoch 3/100
46/46 [==============================] - 0s 480us/step - loss: 0.4799 - accuracy: 0.6957 - val_loss: 0.4603 - val_accuracy: 0.7619
Epoch 4/100
46/46 [==============================] - 0s 581us/step - loss: 0.4275 - accuracy: 0.8478 - val_loss: 0.4270 - val_accuracy: 0.8571
Epoch 5/100
46/46 [==============================] - 0s 632us/step - loss: 0.3699 - accuracy: 0.9130 - val_loss: 0.3876 - val_accuracy: 0.8571
Epoch 6/100
46/46 [==============================] - 0s 674us/step - loss: 0.3120 - accuracy: 0.9130 - val_loss: 0.3568 - val_accuracy: 0.8571
Epoch 7/100
46/46 [==============================] - 0s 567us/step - loss: 0.2503 - accuracy: 0.9130

Epoch 58/100
46/46 [==============================] - 0s 717us/step - loss: 0.0095 - accuracy: 1.0000 - val_loss: 0.2871 - val_accuracy: 0.8571
Epoch 59/100
46/46 [==============================] - 0s 532us/step - loss: 0.0093 - accuracy: 1.0000 - val_loss: 0.2899 - val_accuracy: 0.8571
Epoch 60/100
46/46 [==============================] - 0s 449us/step - loss: 0.0091 - accuracy: 1.0000 - val_loss: 0.2941 - val_accuracy: 0.8571
Epoch 61/100
46/46 [==============================] - 0s 391us/step - loss: 0.0087 - accuracy: 1.0000 - val_loss: 0.2923 - val_accuracy: 0.8571
Epoch 62/100
46/46 [==============================] - 0s 440us/step - loss: 0.0085 - accuracy: 1.0000 - val_loss: 0.2890 - val_accuracy: 0.8571
Epoch 63/100
46/46 [==============================] - 0s 402us/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.2977 - val_accuracy: 0.8571
Epoch 64/100
46/46 [==============================] - 0s 476us/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.3127 - val_accuracy:

In [36]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

21/21 [==============================] - 0s 209us/step
combination test accuracy: 85.71%


In [75]:
#### improve neural network on combination data
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [76]:
model1_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [77]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 46 samples, validate on 21 samples
Epoch 1/2000
46/46 [==============================] - 0s 3ms/step - loss: 1.1826 - accuracy: 0.2609 - val_loss: 0.6962 - val_accuracy: 0.7619
Epoch 2/2000
46/46 [==============================] - 0s 456us/step - loss: 0.6400 - accuracy: 0.7826 - val_loss: 0.6122 - val_accuracy: 0.7143
Epoch 3/2000
46/46 [==============================] - 0s 532us/step - loss: 0.5315 - accuracy: 0.8043 - val_loss: 0.5412 - val_accuracy: 0.7143
Epoch 4/2000
46/46 [==============================] - 0s 527us/step - loss: 0.4339 - accuracy: 0.8478 - val_loss: 0.4739 - val_accuracy: 0.7619
Epoch 5/2000
46/46 [==============================] - 0s 572us/step - loss: 0.3644 - accuracy: 0.8913 - val_loss: 0.4244 - val_accuracy: 0.8571
Epoch 6/2000
46/46 [==============================] - 0s 505us/step - loss: 0.3164 - accuracy: 0.8913 - val_loss: 0.3938 - val_accuracy: 0.9048
Epoch 7/2000
46/46 [==============================] - 0s 503us/step - loss: 0.2799 - accuracy:

46/46 [==============================] - 0s 398us/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 0.4457 - val_accuracy: 0.8571
Epoch 58/2000
46/46 [==============================] - 0s 407us/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 0.4630 - val_accuracy: 0.8571
Epoch 59/2000
46/46 [==============================] - 0s 454us/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.4551 - val_accuracy: 0.8571
Epoch 60/2000
46/46 [==============================] - 0s 546us/step - loss: 0.0059 - accuracy: 1.0000 - val_loss: 0.4250 - val_accuracy: 0.8571
Epoch 61/2000
46/46 [==============================] - 0s 439us/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.3987 - val_accuracy: 0.8571
Epoch 62/2000
46/46 [==============================] - 0s 382us/step - loss: 0.0053 - accuracy: 1.0000 - val_loss: 0.3713 - val_accuracy: 0.8571
Epoch 63/2000
46/46 [==============================] - 0s 380us/step - loss: 0.0054 - accuracy: 1.0000 - val_loss: 0.3528 - val_accuracy: 0.8571

46/46 [==============================] - 0s 522us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.5608 - val_accuracy: 0.8571
Epoch 114/2000
46/46 [==============================] - 0s 488us/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.5583 - val_accuracy: 0.8571
Epoch 115/2000
46/46 [==============================] - 0s 405us/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.5524 - val_accuracy: 0.8571
Epoch 116/2000
46/46 [==============================] - 0s 441us/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.5519 - val_accuracy: 0.8571
Epoch 117/2000
46/46 [==============================] - 0s 468us/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.5533 - val_accuracy: 0.8571
Epoch 118/2000
46/46 [==============================] - 0s 481us/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.5553 - val_accuracy: 0.8571
Epoch 119/2000
46/46 [==============================] - 0s 451us/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.5553 - val_accuracy: 

46/46 [==============================] - 0s 419us/step - loss: 6.0299e-04 - accuracy: 1.0000 - val_loss: 0.6404 - val_accuracy: 0.8571
Epoch 169/2000
46/46 [==============================] - 0s 448us/step - loss: 5.9667e-04 - accuracy: 1.0000 - val_loss: 0.6472 - val_accuracy: 0.8571
Epoch 170/2000
46/46 [==============================] - 0s 399us/step - loss: 5.9244e-04 - accuracy: 1.0000 - val_loss: 0.6534 - val_accuracy: 0.8571
Epoch 171/2000
46/46 [==============================] - 0s 448us/step - loss: 5.8694e-04 - accuracy: 1.0000 - val_loss: 0.6576 - val_accuracy: 0.8571
Epoch 172/2000
46/46 [==============================] - 0s 391us/step - loss: 5.8154e-04 - accuracy: 1.0000 - val_loss: 0.6599 - val_accuracy: 0.8571
Epoch 173/2000
46/46 [==============================] - 0s 430us/step - loss: 5.7581e-04 - accuracy: 1.0000 - val_loss: 0.6607 - val_accuracy: 0.8571
Epoch 174/2000
46/46 [==============================] - 0s 486us/step - loss: 5.6996e-04 - accuracy: 1.0000 - val_l

Epoch 223/2000
46/46 [==============================] - 0s 408us/step - loss: 3.4795e-04 - accuracy: 1.0000 - val_loss: 0.6842 - val_accuracy: 0.8571
Epoch 224/2000
46/46 [==============================] - 0s 378us/step - loss: 3.4475e-04 - accuracy: 1.0000 - val_loss: 0.6857 - val_accuracy: 0.8571
Epoch 225/2000
46/46 [==============================] - 0s 527us/step - loss: 3.4123e-04 - accuracy: 1.0000 - val_loss: 0.6876 - val_accuracy: 0.8571
Epoch 226/2000
46/46 [==============================] - 0s 456us/step - loss: 3.3787e-04 - accuracy: 1.0000 - val_loss: 0.6910 - val_accuracy: 0.8571
Epoch 227/2000
46/46 [==============================] - 0s 395us/step - loss: 3.3500e-04 - accuracy: 1.0000 - val_loss: 0.6935 - val_accuracy: 0.8571
Epoch 228/2000
46/46 [==============================] - 0s 501us/step - loss: 3.3167e-04 - accuracy: 1.0000 - val_loss: 0.6945 - val_accuracy: 0.8571
Epoch 229/2000
46/46 [==============================] - 0s 444us/step - loss: 3.2931e-04 - accuracy:

...

46/46 [==============================] - 0s 390us/step - loss: 4.1075e-06 - accuracy: 1.0000 - val_loss: 1.1701 - val_accuracy: 0.8571
Epoch 1743/2000
46/46 [==============================] - 0s 474us/step - loss: 4.1023e-06 - accuracy: 1.0000 - val_loss: 1.1698 - val_accuracy: 0.8571
Epoch 1744/2000
46/46 [==============================] - 0s 360us/step - loss: 4.0997e-06 - accuracy: 1.0000 - val_loss: 1.1697 - val_accuracy: 0.8571
Epoch 1745/2000
46/46 [==============================] - 0s 422us/step - loss: 4.0971e-06 - accuracy: 1.0000 - val_loss: 1.1699 - val_accuracy: 0.8571
Epoch 1746/2000
46/46 [==============================] - 0s 464us/step - loss: 4.0894e-06 - accuracy: 1.0000 - val_loss: 1.1701 - val_accuracy: 0.8571
Epoch 1747/2000
46/46 [==============================] - 0s 479us/step - loss: 4.0842e-06 - accuracy: 1.0000 - val_loss: 1.1705 - val_accuracy: 0.8571
Epoch 1748/2000
46/46 [==============================] - 0s 389us/step - loss: 4.0790e-06 - accuracy: 1.0000 -

46/46 [==============================] - 0s 392us/step - loss: 3.8043e-06 - accuracy: 1.0000 - val_loss: 1.1821 - val_accuracy: 0.8571
Epoch 1797/2000
46/46 [==============================] - 0s 402us/step - loss: 3.7991e-06 - accuracy: 1.0000 - val_loss: 1.1822 - val_accuracy: 0.8571
Epoch 1798/2000
46/46 [==============================] - 0s 471us/step - loss: 3.7913e-06 - accuracy: 1.0000 - val_loss: 1.1825 - val_accuracy: 0.8571
Epoch 1799/2000
46/46 [==============================] - 0s 429us/step - loss: 3.7836e-06 - accuracy: 1.0000 - val_loss: 1.1828 - val_accuracy: 0.8571
Epoch 1800/2000
46/46 [==============================] - 0s 465us/step - loss: 3.7784e-06 - accuracy: 1.0000 - val_loss: 1.1830 - val_accuracy: 0.8571
Epoch 1801/2000
46/46 [==============================] - 0s 393us/step - loss: 3.7784e-06 - accuracy: 1.0000 - val_loss: 1.1833 - val_accuracy: 0.8571
Epoch 1802/2000
46/46 [==============================] - 0s 443us/step - loss: 3.7758e-06 - accuracy: 1.0000 -

46/46 [==============================] - 0s 434us/step - loss: 3.5296e-06 - accuracy: 1.0000 - val_loss: 1.1899 - val_accuracy: 0.8571
Epoch 1851/2000
46/46 [==============================] - 0s 463us/step - loss: 3.5192e-06 - accuracy: 1.0000 - val_loss: 1.1900 - val_accuracy: 0.8571
Epoch 1852/2000
46/46 [==============================] - 0s 356us/step - loss: 3.5192e-06 - accuracy: 1.0000 - val_loss: 1.1904 - val_accuracy: 0.8571
Epoch 1853/2000
46/46 [==============================] - 0s 387us/step - loss: 3.5166e-06 - accuracy: 1.0000 - val_loss: 1.1909 - val_accuracy: 0.8571
Epoch 1854/2000
46/46 [==============================] - 0s 367us/step - loss: 3.5063e-06 - accuracy: 1.0000 - val_loss: 1.1909 - val_accuracy: 0.8571
Epoch 1855/2000
46/46 [==============================] - 0s 372us/step - loss: 3.5011e-06 - accuracy: 1.0000 - val_loss: 1.1911 - val_accuracy: 0.8571
Epoch 1856/2000
46/46 [==============================] - 0s 332us/step - loss: 3.4881e-06 - accuracy: 1.0000 -

46/46 [==============================] - 0s 357us/step - loss: 3.2523e-06 - accuracy: 1.0000 - val_loss: 1.2021 - val_accuracy: 0.8571
Epoch 1905/2000
46/46 [==============================] - 0s 412us/step - loss: 3.2419e-06 - accuracy: 1.0000 - val_loss: 1.2018 - val_accuracy: 0.8571
Epoch 1906/2000
46/46 [==============================] - 0s 434us/step - loss: 3.2394e-06 - accuracy: 1.0000 - val_loss: 1.2017 - val_accuracy: 0.8571
Epoch 1907/2000
46/46 [==============================] - 0s 379us/step - loss: 3.2264e-06 - accuracy: 1.0000 - val_loss: 1.2018 - val_accuracy: 0.8571
Epoch 1908/2000
46/46 [==============================] - 0s 346us/step - loss: 3.2238e-06 - accuracy: 1.0000 - val_loss: 1.2019 - val_accuracy: 0.8571
Epoch 1909/2000
46/46 [==============================] - 0s 388us/step - loss: 3.2186e-06 - accuracy: 1.0000 - val_loss: 1.2019 - val_accuracy: 0.8571
Epoch 1910/2000
46/46 [==============================] - 0s 468us/step - loss: 3.2160e-06 - accuracy: 1.0000 -

46/46 [==============================] - 0s 447us/step - loss: 3.0061e-06 - accuracy: 1.0000 - val_loss: 1.2106 - val_accuracy: 0.8571
Epoch 1959/2000
46/46 [==============================] - 0s 661us/step - loss: 3.0009e-06 - accuracy: 1.0000 - val_loss: 1.2107 - val_accuracy: 0.8571
Epoch 1960/2000
46/46 [==============================] - 0s 369us/step - loss: 2.9958e-06 - accuracy: 1.0000 - val_loss: 1.2110 - val_accuracy: 0.8571
Epoch 1961/2000
46/46 [==============================] - 0s 416us/step - loss: 2.9932e-06 - accuracy: 1.0000 - val_loss: 1.2115 - val_accuracy: 0.8571
Epoch 1962/2000
46/46 [==============================] - 0s 398us/step - loss: 2.9880e-06 - accuracy: 1.0000 - val_loss: 1.2119 - val_accuracy: 0.8571
Epoch 1963/2000
46/46 [==============================] - 0s 435us/step - loss: 2.9828e-06 - accuracy: 1.0000 - val_loss: 1.2120 - val_accuracy: 0.8571
Epoch 1964/2000
46/46 [==============================] - 0s 425us/step - loss: 2.9802e-06 - accuracy: 1.0000 -

In [80]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

21/21 [==============================] - 0s 275us/step
combination test accuracy: 85.71%


In [79]:
############ Feature selection using lasso ##########

In [81]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [82]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [83]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [84]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [85]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 7002
selected features: 112


In [86]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 148,  342,  425,  462,  488,  534,  570,  802,  803,  908,  952,
         955, 1154, 1201, 1231, 1290, 1343, 1479, 1480, 1536, 1671, 1925,
        2034, 2088, 2104, 2122, 2240, 2318, 2320, 2373, 2563, 2587, 2594,
        2614, 2618, 2643, 2759, 2790, 2804, 2807, 2815, 2816, 2830, 2860,
        2997, 3102, 3109, 3151, 3153, 3173, 3354, 3355, 3359, 3507, 3584,
        3686, 3722, 3825, 3837, 3862, 3946, 3965, 4027, 4059, 4294, 4331,
        4351, 4458, 4474, 4504, 4552, 4568, 4586, 4623, 4736, 4762, 4813,
        4939, 5004, 5051, 5100, 5167, 5189, 5217, 5339, 5363, 5382, 5408,
        5428, 5454, 5455, 5499, 5631, 5639, 5690, 5707, 5758, 5831, 5913,
        5972, 6231, 6251, 6547, 6596, 6642, 6646, 6691, 6692, 6694, 6745,
        6746, 6935]])

In [87]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTGGTGTTCGGAGATAAATATATCGGCGCTTCTCTACTTTCAATATTAGCTGAAGCATATGAATCGATAATAATTCGATTGTAGTCAGATGTTAAAGC',
       'TTTGGTGTTCGGAGATAAATATATCGGCGCTTCTCTACTTTCAATATTAGCTGAAGCATATGAATCGATAATAATTCGATTGTAGTCAGATGTTAAAGCG',
       'TTTCTTGATTTTTA',
       'TTTCGTATGCACTATCAGCGTTTGATAATGTGAATGCGAATCGGTTTAAACCAGGTTTATTTTTGGTGTTCGGAGATAAATATATCGGCGCTTCTCTACT',
       'TTTCCATTTATTTC',
       'TTTATTTTTGGTGTTCGGAGATAAATATATCGGCGCTTCTCTACTTTCAATATTAGCTGAAGCATATGAATCGATAATAATTCGATTGTAGTCAGATGTT',
       'TTTATCTATATCA', 'TTGTATATAAATA', 'TTGTATATAAATATATATC',
       'TTGCCATTCACTCATTAACTCGT', 'TTGATAGTAAAGG',
       'TTGATAATGTGAATGCGAATCGGTTTAAACCAGGTTTATTTTTGGTGTTCGGAGATAAATATATCGGCGCTTCTCTACTTTCAATATTAGCTGAAGCATA',
       'TTCCATTTATTTC',
       'TTCATATGCTTCAGCTAATATTGAAAGTAGAGAAGCGCCGATATATTTATCTCCGAACACCAAAAATAAACCTGGTTTAAACCGATTCGCATTCACATTA',
       'TTCACTCATTAACTCGTTATAACTCATACGATTATTCTCATTTTGTAGTAATTTAGCAAAGCCAATATCATTCGCATCTTGTTTTAACATATGACTCAAT',
       'TTATTTTATTACTA',
       'T

In [89]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 112) (255,) (255, 113)


In [90]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 92), (1, 92), (2, 92)]


In [91]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 20), (1, 18), (2, 32)]


In [92]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [93]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [98]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(3, activation='softmax'),
])

In [99]:
model2_over.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [101]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=100,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 193 samples, validate on 83 samples
Epoch 1/100
193/193 [==============================] - 0s 111us/step - loss: 0.3473 - accuracy: 0.8497 - val_loss: 1.3802 - val_accuracy: 0.5783
Epoch 2/100
193/193 [==============================] - 0s 113us/step - loss: 0.3509 - accuracy: 0.8187 - val_loss: 1.3831 - val_accuracy: 0.5663
Epoch 3/100
193/193 [==============================] - 0s 146us/step - loss: 0.3523 - accuracy: 0.8187 - val_loss: 1.3896 - val_accuracy: 0.5663
Epoch 4/100
193/193 [==============================] - 0s 145us/step - loss: 0.3524 - accuracy: 0.8083 - val_loss: 1.3922 - val_accuracy: 0.5663
Epoch 5/100
193/193 [==============================] - 0s 177us/step - loss: 0.3508 - accuracy: 0.8497 - val_loss: 1.3917 - val_accuracy: 0.5783
Epoch 6/100
193/193 [==============================] - 0s 110us/step - loss: 0.3480 - accuracy: 0.8497 - val_loss: 1.3914 - val_accuracy: 0.5663
Epoch 7/100
193/193 [==============================] - 0s 143us/step - loss: 0.3467 -

Epoch 57/100
193/193 [==============================] - ETA: 0s - loss: 0.4164 - accuracy: 0.81 - 0s 124us/step - loss: 0.3354 - accuracy: 0.8549 - val_loss: 1.4185 - val_accuracy: 0.5904
Epoch 58/100
193/193 [==============================] - 0s 121us/step - loss: 0.3360 - accuracy: 0.8549 - val_loss: 1.4226 - val_accuracy: 0.5904
Epoch 59/100
193/193 [==============================] - 0s 111us/step - loss: 0.3351 - accuracy: 0.8549 - val_loss: 1.4278 - val_accuracy: 0.5904
Epoch 60/100
193/193 [==============================] - 0s 104us/step - loss: 0.3340 - accuracy: 0.8446 - val_loss: 1.4343 - val_accuracy: 0.5783
Epoch 61/100
193/193 [==============================] - 0s 100us/step - loss: 0.3340 - accuracy: 0.8549 - val_loss: 1.4379 - val_accuracy: 0.5783
Epoch 62/100
193/193 [==============================] - 0s 108us/step - loss: 0.3353 - accuracy: 0.8497 - val_loss: 1.4347 - val_accuracy: 0.5783
Epoch 63/100
193/193 [==============================] - 0s 118us/step - loss: 0.34

In [102]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

83/83 [==============================] - 0s 105us/step
combination test accuracy: 57.83%


In [124]:
#### improve neural network on over-sampling data
model2_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [125]:
model2_over2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [126]:
model2_over2.fit(X_sel_train_over, y_sel_train_over,
          batch_size=16, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 193 samples, validate on 83 samples
Epoch 1/1000
193/193 [==============================] - 0s 717us/step - loss: 1.1574 - accuracy: 0.3109 - val_loss: 1.0318 - val_accuracy: 0.4096
Epoch 2/1000
193/193 [==============================] - 0s 182us/step - loss: 1.0390 - accuracy: 0.4145 - val_loss: 1.0083 - val_accuracy: 0.4096
Epoch 3/1000
193/193 [==============================] - 0s 197us/step - loss: 1.0012 - accuracy: 0.4663 - val_loss: 0.9765 - val_accuracy: 0.4578
Epoch 4/1000
193/193 [==============================] - 0s 181us/step - loss: 0.9756 - accuracy: 0.5078 - val_loss: 0.9561 - val_accuracy: 0.5181
Epoch 5/1000
193/193 [==============================] - 0s 185us/step - loss: 0.9519 - accuracy: 0.5907 - val_loss: 0.9376 - val_accuracy: 0.5663
Epoch 6/1000
193/193 [==============================] - 0s 203us/step - loss: 0.9252 - accuracy: 0.6269 - val_loss: 0.9234 - val_accuracy: 0.5542
Epoch 7/1000
193/193 [==============================] - 0s 183us/step - loss: 0

193/193 [==============================] - 0s 184us/step - loss: 0.5985 - accuracy: 0.7927 - val_loss: 0.8952 - val_accuracy: 0.6265
Epoch 57/1000
193/193 [==============================] - 0s 185us/step - loss: 0.5914 - accuracy: 0.7979 - val_loss: 0.8850 - val_accuracy: 0.6145
Epoch 58/1000
193/193 [==============================] - 0s 163us/step - loss: 0.5895 - accuracy: 0.7979 - val_loss: 0.8781 - val_accuracy: 0.6145
Epoch 59/1000
193/193 [==============================] - 0s 153us/step - loss: 0.5939 - accuracy: 0.7979 - val_loss: 0.8806 - val_accuracy: 0.6145
Epoch 60/1000
193/193 [==============================] - 0s 172us/step - loss: 0.5909 - accuracy: 0.8031 - val_loss: 0.8808 - val_accuracy: 0.6145
Epoch 61/1000
193/193 [==============================] - 0s 151us/step - loss: 0.5923 - accuracy: 0.7824 - val_loss: 0.8754 - val_accuracy: 0.6024
Epoch 62/1000
193/193 [==============================] - 0s 164us/step - loss: 0.5755 - accuracy: 0.8083 - val_loss: 0.8825 - val_ac

Epoch 112/1000
193/193 [==============================] - 0s 148us/step - loss: 0.4658 - accuracy: 0.8187 - val_loss: 0.9937 - val_accuracy: 0.6386
Epoch 113/1000
193/193 [==============================] - 0s 142us/step - loss: 0.4794 - accuracy: 0.8135 - val_loss: 1.0017 - val_accuracy: 0.6386
Epoch 114/1000
193/193 [==============================] - 0s 150us/step - loss: 0.4751 - accuracy: 0.8135 - val_loss: 0.9904 - val_accuracy: 0.6265
Epoch 115/1000
193/193 [==============================] - 0s 153us/step - loss: 0.4552 - accuracy: 0.8135 - val_loss: 1.0011 - val_accuracy: 0.6386
Epoch 116/1000
193/193 [==============================] - 0s 149us/step - loss: 0.4441 - accuracy: 0.8187 - val_loss: 1.0071 - val_accuracy: 0.6386
Epoch 117/1000
193/193 [==============================] - 0s 145us/step - loss: 0.5511 - accuracy: 0.7824 - val_loss: 1.0039 - val_accuracy: 0.6386
Epoch 118/1000
193/193 [==============================] - 0s 136us/step - loss: 0.4795 - accuracy: 0.8031 - val_

193/193 [==============================] - 0s 149us/step - loss: 0.3984 - accuracy: 0.8342 - val_loss: 1.0975 - val_accuracy: 0.6265
Epoch 168/1000
193/193 [==============================] - 0s 156us/step - loss: 0.4108 - accuracy: 0.8342 - val_loss: 1.1344 - val_accuracy: 0.6265
Epoch 169/1000
193/193 [==============================] - 0s 143us/step - loss: 0.4140 - accuracy: 0.8135 - val_loss: 1.0880 - val_accuracy: 0.6506
Epoch 170/1000
193/193 [==============================] - 0s 149us/step - loss: 0.4082 - accuracy: 0.8187 - val_loss: 1.0857 - val_accuracy: 0.6386
Epoch 171/1000
193/193 [==============================] - 0s 146us/step - loss: 0.3913 - accuracy: 0.8342 - val_loss: 1.1025 - val_accuracy: 0.6386
Epoch 172/1000
193/193 [==============================] - 0s 150us/step - loss: 0.3850 - accuracy: 0.8394 - val_loss: 1.1049 - val_accuracy: 0.6386
Epoch 173/1000
193/193 [==============================] - 0s 142us/step - loss: 0.3878 - accuracy: 0.8342 - val_loss: 1.1111 - 

193/193 [==============================] - 0s 150us/step - loss: 0.3877 - accuracy: 0.8342 - val_loss: 1.1920 - val_accuracy: 0.6265
Epoch 223/1000
193/193 [==============================] - 0s 178us/step - loss: 0.3700 - accuracy: 0.8238 - val_loss: 1.2240 - val_accuracy: 0.6506
Epoch 224/1000
193/193 [==============================] - 0s 148us/step - loss: 0.3630 - accuracy: 0.8342 - val_loss: 1.2304 - val_accuracy: 0.6627
Epoch 225/1000
193/193 [==============================] - 0s 147us/step - loss: 0.3601 - accuracy: 0.8446 - val_loss: 1.2028 - val_accuracy: 0.6386
Epoch 226/1000
193/193 [==============================] - 0s 140us/step - loss: 0.3580 - accuracy: 0.8497 - val_loss: 1.2157 - val_accuracy: 0.6145
Epoch 227/1000
193/193 [==============================] - 0s 142us/step - loss: 0.3628 - accuracy: 0.8446 - val_loss: 1.2498 - val_accuracy: 0.5904
Epoch 228/1000
193/193 [==============================] - 0s 155us/step - loss: 0.3615 - accuracy: 0.8446 - val_loss: 1.2510 - 

...

193/193 [==============================] - 0s 154us/step - loss: 0.2933 - accuracy: 0.8549 - val_loss: 2.2930 - val_accuracy: 0.6386
Epoch 773/1000
193/193 [==============================] - 0s 148us/step - loss: 0.2935 - accuracy: 0.8549 - val_loss: 2.3706 - val_accuracy: 0.6265
Epoch 774/1000
193/193 [==============================] - 0s 139us/step - loss: 0.2898 - accuracy: 0.8549 - val_loss: 2.3651 - val_accuracy: 0.6265
Epoch 775/1000
193/193 [==============================] - 0s 146us/step - loss: 0.2911 - accuracy: 0.8394 - val_loss: 2.3714 - val_accuracy: 0.6145
Epoch 776/1000
193/193 [==============================] - 0s 150us/step - loss: 0.2899 - accuracy: 0.8549 - val_loss: 2.3798 - val_accuracy: 0.6265
Epoch 777/1000
193/193 [==============================] - 0s 145us/step - loss: 0.2898 - accuracy: 0.8549 - val_loss: 2.3532 - val_accuracy: 0.6145
Epoch 778/1000
193/193 [==============================] - 0s 226us/step - loss: 0.2965 - accuracy: 0.8497 - val_loss: 2.2743 - 

193/193 [==============================] - 0s 152us/step - loss: 0.2891 - accuracy: 0.8549 - val_loss: 2.4919 - val_accuracy: 0.6024
Epoch 828/1000
193/193 [==============================] - 0s 156us/step - loss: 0.2899 - accuracy: 0.8601 - val_loss: 2.4218 - val_accuracy: 0.6145
Epoch 829/1000
193/193 [==============================] - 0s 153us/step - loss: 0.2945 - accuracy: 0.8549 - val_loss: 2.4292 - val_accuracy: 0.6145
Epoch 830/1000
193/193 [==============================] - 0s 141us/step - loss: 0.2901 - accuracy: 0.8601 - val_loss: 2.4800 - val_accuracy: 0.6145
Epoch 831/1000
193/193 [==============================] - 0s 147us/step - loss: 0.2904 - accuracy: 0.8497 - val_loss: 2.4981 - val_accuracy: 0.6145
Epoch 832/1000
193/193 [==============================] - 0s 144us/step - loss: 0.2910 - accuracy: 0.8601 - val_loss: 2.5230 - val_accuracy: 0.6024
Epoch 833/1000
193/193 [==============================] - 0s 155us/step - loss: 0.3049 - accuracy: 0.8394 - val_loss: 2.5194 - 

193/193 [==============================] - 0s 139us/step - loss: 0.2954 - accuracy: 0.8497 - val_loss: 2.5964 - val_accuracy: 0.5904
Epoch 883/1000
193/193 [==============================] - 0s 153us/step - loss: 0.2942 - accuracy: 0.8549 - val_loss: 2.6358 - val_accuracy: 0.5904
Epoch 884/1000
193/193 [==============================] - 0s 155us/step - loss: 0.2951 - accuracy: 0.8601 - val_loss: 2.6375 - val_accuracy: 0.5904
Epoch 885/1000
193/193 [==============================] - 0s 151us/step - loss: 0.2886 - accuracy: 0.8601 - val_loss: 2.6506 - val_accuracy: 0.6024
Epoch 886/1000
193/193 [==============================] - 0s 142us/step - loss: 0.2904 - accuracy: 0.8549 - val_loss: 2.6270 - val_accuracy: 0.6024
Epoch 887/1000
193/193 [==============================] - 0s 145us/step - loss: 0.2859 - accuracy: 0.8446 - val_loss: 2.6237 - val_accuracy: 0.6024
Epoch 888/1000
193/193 [==============================] - 0s 158us/step - loss: 0.2865 - accuracy: 0.8549 - val_loss: 2.6116 - 

193/193 [==============================] - 0s 154us/step - loss: 0.2873 - accuracy: 0.8549 - val_loss: 2.6303 - val_accuracy: 0.6265
Epoch 938/1000
193/193 [==============================] - 0s 156us/step - loss: 0.2910 - accuracy: 0.8446 - val_loss: 2.6549 - val_accuracy: 0.6145
Epoch 939/1000
193/193 [==============================] - 0s 153us/step - loss: 0.2891 - accuracy: 0.8601 - val_loss: 2.6904 - val_accuracy: 0.6145
Epoch 940/1000
193/193 [==============================] - 0s 138us/step - loss: 0.2903 - accuracy: 0.8394 - val_loss: 2.6784 - val_accuracy: 0.6265
Epoch 941/1000
193/193 [==============================] - 0s 145us/step - loss: 0.3048 - accuracy: 0.8394 - val_loss: 2.7221 - val_accuracy: 0.6145
Epoch 942/1000
193/193 [==============================] - 0s 149us/step - loss: 0.2985 - accuracy: 0.8187 - val_loss: 2.8569 - val_accuracy: 0.5904
Epoch 943/1000
193/193 [==============================] - 0s 162us/step - loss: 0.3274 - accuracy: 0.8497 - val_loss: 2.6758 - 

193/193 [==============================] - 0s 149us/step - loss: 0.3931 - accuracy: 0.8394 - val_loss: 2.7278 - val_accuracy: 0.5904
Epoch 993/1000
193/193 [==============================] - 0s 158us/step - loss: 0.4121 - accuracy: 0.8497 - val_loss: 2.5704 - val_accuracy: 0.6145
Epoch 994/1000
193/193 [==============================] - 0s 164us/step - loss: 0.3702 - accuracy: 0.8446 - val_loss: 2.5932 - val_accuracy: 0.5904
Epoch 995/1000
193/193 [==============================] - 0s 153us/step - loss: 0.3699 - accuracy: 0.8394 - val_loss: 2.7996 - val_accuracy: 0.5904
Epoch 996/1000
193/193 [==============================] - 0s 150us/step - loss: 0.3216 - accuracy: 0.8549 - val_loss: 2.6364 - val_accuracy: 0.6265
Epoch 997/1000
193/193 [==============================] - 0s 149us/step - loss: 0.3121 - accuracy: 0.8549 - val_loss: 2.5063 - val_accuracy: 0.6386
Epoch 998/1000
193/193 [==============================] - 0s 143us/step - loss: 0.3051 - accuracy: 0.8549 - val_loss: 2.5438 - 

In [128]:
acc_test_over3 = model2_over2.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over3*100))

83/83 [==============================] - 0s 198us/step
combination test accuracy: 65.06%


In [129]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(3, activation='softmax'),
])

In [130]:
model2_comb.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [131]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 49 samples, validate on 21 samples
Epoch 1/1000
49/49 [==============================] - 0s 2ms/step - loss: 1.0301 - accuracy: 0.5714 - val_loss: 0.8826 - val_accuracy: 0.5714
Epoch 2/1000
49/49 [==============================] - 0s 164us/step - loss: 0.8770 - accuracy: 0.5714 - val_loss: 0.7865 - val_accuracy: 0.6190
Epoch 3/1000
49/49 [==============================] - 0s 203us/step - loss: 0.7801 - accuracy: 0.5714 - val_loss: 0.7237 - val_accuracy: 0.6667
Epoch 4/1000
49/49 [==============================] - 0s 241us/step - loss: 0.7174 - accuracy: 0.6122 - val_loss: 0.6771 - val_accuracy: 0.7143
Epoch 5/1000
49/49 [==============================] - 0s 225us/step - loss: 0.6725 - accuracy: 0.6327 - val_loss: 0.6398 - val_accuracy: 0.7143
Epoch 6/1000
49/49 [==============================] - 0s 257us/step - loss: 0.6346 - accuracy: 0.6327 - val_loss: 0.6088 - val_accuracy: 0.7143
Epoch 7/1000
49/49 [==============================] - 0s 255us/step - loss: 0.6032 - accuracy:

49/49 [==============================] - 0s 184us/step - loss: 0.0883 - accuracy: 1.0000 - val_loss: 0.3620 - val_accuracy: 0.8571
Epoch 58/1000
49/49 [==============================] - 0s 192us/step - loss: 0.0856 - accuracy: 1.0000 - val_loss: 0.3623 - val_accuracy: 0.8571
Epoch 59/1000
49/49 [==============================] - 0s 202us/step - loss: 0.0827 - accuracy: 1.0000 - val_loss: 0.3627 - val_accuracy: 0.8571
Epoch 60/1000
49/49 [==============================] - 0s 209us/step - loss: 0.0801 - accuracy: 1.0000 - val_loss: 0.3629 - val_accuracy: 0.8571
Epoch 61/1000
49/49 [==============================] - 0s 385us/step - loss: 0.0777 - accuracy: 1.0000 - val_loss: 0.3633 - val_accuracy: 0.8571
Epoch 62/1000
49/49 [==============================] - 0s 200us/step - loss: 0.0750 - accuracy: 1.0000 - val_loss: 0.3638 - val_accuracy: 0.8571
Epoch 63/1000
49/49 [==============================] - 0s 168us/step - loss: 0.0727 - accuracy: 1.0000 - val_loss: 0.3643 - val_accuracy: 0.8571

49/49 [==============================] - 0s 328us/step - loss: 0.0202 - accuracy: 1.0000 - val_loss: 0.4189 - val_accuracy: 0.8571
Epoch 114/1000
49/49 [==============================] - 0s 206us/step - loss: 0.0198 - accuracy: 1.0000 - val_loss: 0.4199 - val_accuracy: 0.8571
Epoch 115/1000
49/49 [==============================] - 0s 357us/step - loss: 0.0194 - accuracy: 1.0000 - val_loss: 0.4209 - val_accuracy: 0.8571
Epoch 116/1000
49/49 [==============================] - 0s 276us/step - loss: 0.0190 - accuracy: 1.0000 - val_loss: 0.4220 - val_accuracy: 0.8571
Epoch 117/1000
49/49 [==============================] - 0s 222us/step - loss: 0.0187 - accuracy: 1.0000 - val_loss: 0.4231 - val_accuracy: 0.8571
Epoch 118/1000
49/49 [==============================] - 0s 185us/step - loss: 0.0184 - accuracy: 1.0000 - val_loss: 0.4244 - val_accuracy: 0.8571
Epoch 119/1000
49/49 [==============================] - 0s 176us/step - loss: 0.0180 - accuracy: 1.0000 - val_loss: 0.4260 - val_accuracy: 

49/49 [==============================] - 0s 199us/step - loss: 0.0086 - accuracy: 1.0000 - val_loss: 0.4824 - val_accuracy: 0.8571
Epoch 170/1000
49/49 [==============================] - 0s 166us/step - loss: 0.0085 - accuracy: 1.0000 - val_loss: 0.4834 - val_accuracy: 0.8571
Epoch 171/1000
49/49 [==============================] - 0s 164us/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 0.4843 - val_accuracy: 0.8571
Epoch 172/1000
49/49 [==============================] - 0s 160us/step - loss: 0.0083 - accuracy: 1.0000 - val_loss: 0.4853 - val_accuracy: 0.8571
Epoch 173/1000
49/49 [==============================] - 0s 227us/step - loss: 0.0082 - accuracy: 1.0000 - val_loss: 0.4862 - val_accuracy: 0.8571
Epoch 174/1000
49/49 [==============================] - 0s 188us/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.4872 - val_accuracy: 0.8571
Epoch 175/1000
49/49 [==============================] - 0s 179us/step - loss: 0.0080 - accuracy: 1.0000 - val_loss: 0.4882 - val_accuracy: 

49/49 [==============================] - 0s 194us/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.5255 - val_accuracy: 0.8571
Epoch 226/1000
49/49 [==============================] - 0s 207us/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.5262 - val_accuracy: 0.8571
Epoch 227/1000
49/49 [==============================] - 0s 199us/step - loss: 0.0046 - accuracy: 1.0000 - val_loss: 0.5270 - val_accuracy: 0.8571
Epoch 228/1000
49/49 [==============================] - 0s 227us/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.5277 - val_accuracy: 0.8571
Epoch 229/1000
49/49 [==============================] - 0s 207us/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 0.5283 - val_accuracy: 0.8571
Epoch 230/1000
49/49 [==============================] - 0s 208us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.5290 - val_accuracy: 0.8571
Epoch 231/1000
49/49 [==============================] - 0s 161us/step - loss: 0.0044 - accuracy: 1.0000 - val_loss: 0.5298 - val_accuracy: 

...

49/49 [==============================] - 0s 148us/step - loss: 3.4245e-04 - accuracy: 1.0000 - val_loss: 0.7426 - val_accuracy: 0.8571
Epoch 776/1000
49/49 [==============================] - 0s 162us/step - loss: 3.4139e-04 - accuracy: 1.0000 - val_loss: 0.7427 - val_accuracy: 0.8571
Epoch 777/1000
49/49 [==============================] - 0s 181us/step - loss: 3.4049e-04 - accuracy: 1.0000 - val_loss: 0.7429 - val_accuracy: 0.8571
Epoch 778/1000
49/49 [==============================] - 0s 155us/step - loss: 3.3948e-04 - accuracy: 1.0000 - val_loss: 0.7430 - val_accuracy: 0.8571
Epoch 779/1000
49/49 [==============================] - 0s 199us/step - loss: 3.3837e-04 - accuracy: 1.0000 - val_loss: 0.7432 - val_accuracy: 0.8571
Epoch 780/1000
49/49 [==============================] - 0s 400us/step - loss: 3.3766e-04 - accuracy: 1.0000 - val_loss: 0.7433 - val_accuracy: 0.8571
Epoch 781/1000
49/49 [==============================] - 0s 206us/step - loss: 3.3649e-04 - accuracy: 1.0000 - val_l

Epoch 830/1000
49/49 [==============================] - 0s 215us/step - loss: 2.9277e-04 - accuracy: 1.0000 - val_loss: 0.7546 - val_accuracy: 0.8571
Epoch 831/1000
49/49 [==============================] - 0s 257us/step - loss: 2.9206e-04 - accuracy: 1.0000 - val_loss: 0.7549 - val_accuracy: 0.8571
Epoch 832/1000
49/49 [==============================] - 0s 253us/step - loss: 2.9119e-04 - accuracy: 1.0000 - val_loss: 0.7551 - val_accuracy: 0.8571
Epoch 833/1000
49/49 [==============================] - 0s 230us/step - loss: 2.9044e-04 - accuracy: 1.0000 - val_loss: 0.7553 - val_accuracy: 0.8571
Epoch 834/1000
49/49 [==============================] - 0s 221us/step - loss: 2.8965e-04 - accuracy: 1.0000 - val_loss: 0.7556 - val_accuracy: 0.8571
Epoch 835/1000
49/49 [==============================] - 0s 174us/step - loss: 2.8890e-04 - accuracy: 1.0000 - val_loss: 0.7559 - val_accuracy: 0.8571
Epoch 836/1000
49/49 [==============================] - 0s 199us/step - loss: 2.8808e-04 - accuracy:

49/49 [==============================] - 0s 216us/step - loss: 2.5372e-04 - accuracy: 1.0000 - val_loss: 0.7666 - val_accuracy: 0.8571
Epoch 885/1000
49/49 [==============================] - 0s 198us/step - loss: 2.5304e-04 - accuracy: 1.0000 - val_loss: 0.7668 - val_accuracy: 0.8571
Epoch 886/1000
49/49 [==============================] - 0s 197us/step - loss: 2.5249e-04 - accuracy: 1.0000 - val_loss: 0.7671 - val_accuracy: 0.8571
Epoch 887/1000
49/49 [==============================] - 0s 197us/step - loss: 2.5179e-04 - accuracy: 1.0000 - val_loss: 0.7673 - val_accuracy: 0.8571
Epoch 888/1000
49/49 [==============================] - 0s 252us/step - loss: 2.5120e-04 - accuracy: 1.0000 - val_loss: 0.7675 - val_accuracy: 0.8571
Epoch 889/1000
49/49 [==============================] - 0s 246us/step - loss: 2.5055e-04 - accuracy: 1.0000 - val_loss: 0.7677 - val_accuracy: 0.8571
Epoch 890/1000
49/49 [==============================] - 0s 232us/step - loss: 2.4987e-04 - accuracy: 1.0000 - val_l

49/49 [==============================] - 0s 239us/step - loss: 2.2094e-04 - accuracy: 1.0000 - val_loss: 0.7796 - val_accuracy: 0.8571
Epoch 939/1000
49/49 [==============================] - 0s 206us/step - loss: 2.2031e-04 - accuracy: 1.0000 - val_loss: 0.7798 - val_accuracy: 0.8571
Epoch 940/1000
49/49 [==============================] - 0s 222us/step - loss: 2.1977e-04 - accuracy: 1.0000 - val_loss: 0.7799 - val_accuracy: 0.8571
Epoch 941/1000
49/49 [==============================] - 0s 248us/step - loss: 2.1928e-04 - accuracy: 1.0000 - val_loss: 0.7801 - val_accuracy: 0.8571
Epoch 942/1000
49/49 [==============================] - 0s 229us/step - loss: 2.1870e-04 - accuracy: 1.0000 - val_loss: 0.7803 - val_accuracy: 0.8571
Epoch 943/1000
49/49 [==============================] - 0s 204us/step - loss: 2.1807e-04 - accuracy: 1.0000 - val_loss: 0.7805 - val_accuracy: 0.8571
Epoch 944/1000
49/49 [==============================] - 0s 180us/step - loss: 2.1758e-04 - accuracy: 1.0000 - val_l

Epoch 993/1000
49/49 [==============================] - 0s 197us/step - loss: 1.9305e-04 - accuracy: 1.0000 - val_loss: 0.7901 - val_accuracy: 0.8571
Epoch 994/1000
49/49 [==============================] - 0s 228us/step - loss: 1.9256e-04 - accuracy: 1.0000 - val_loss: 0.7903 - val_accuracy: 0.8571
Epoch 995/1000
49/49 [==============================] - 0s 180us/step - loss: 1.9210e-04 - accuracy: 1.0000 - val_loss: 0.7905 - val_accuracy: 0.8571
Epoch 996/1000
49/49 [==============================] - 0s 198us/step - loss: 1.9159e-04 - accuracy: 1.0000 - val_loss: 0.7907 - val_accuracy: 0.8571
Epoch 997/1000
49/49 [==============================] - 0s 217us/step - loss: 1.9117e-04 - accuracy: 1.0000 - val_loss: 0.7909 - val_accuracy: 0.8571
Epoch 998/1000
49/49 [==============================] - 0s 200us/step - loss: 1.9070e-04 - accuracy: 1.0000 - val_loss: 0.7911 - val_accuracy: 0.8571
Epoch 999/1000
49/49 [==============================] - 0s 251us/step - loss: 1.9021e-04 - accuracy:

In [132]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

21/21 [==============================] - 0s 137us/step
combination test accuracy: 85.71%


In [145]:
#### improve neural network on combination data
model2_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(32, activation='relu'),
    Dense(3, activation='softmax'),
])

In [146]:
model2_comb2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [147]:
model2_comb2.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=64, epochs=2000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 49 samples, validate on 21 samples
Epoch 1/2000
49/49 [==============================] - 0s 2ms/step - loss: 1.0914 - accuracy: 0.0204 - val_loss: 0.9852 - val_accuracy: 0.3333
Epoch 2/2000
49/49 [==============================] - 0s 67us/step - loss: 1.0359 - accuracy: 0.2449 - val_loss: 0.9357 - val_accuracy: 0.4762
Epoch 3/2000
49/49 [==============================] - 0s 101us/step - loss: 0.9788 - accuracy: 0.4490 - val_loss: 0.8887 - val_accuracy: 0.4762
Epoch 4/2000
49/49 [==============================] - 0s 119us/step - loss: 0.9255 - accuracy: 0.4490 - val_loss: 0.8458 - val_accuracy: 0.4762
Epoch 5/2000
49/49 [==============================] - 0s 138us/step - loss: 0.8782 - accuracy: 0.4490 - val_loss: 0.8083 - val_accuracy: 0.4762
Epoch 6/2000
49/49 [==============================] - 0s 153us/step - loss: 0.8374 - accuracy: 0.4490 - val_loss: 0.7780 - val_accuracy: 0.4762
Epoch 7/2000
49/49 [==============================] - 0s 123us/step - loss: 0.8051 - accuracy: 

Epoch 58/2000
49/49 [==============================] - 0s 155us/step - loss: 0.2312 - accuracy: 0.9796 - val_loss: 0.3637 - val_accuracy: 0.8571
Epoch 59/2000
49/49 [==============================] - 0s 159us/step - loss: 0.2250 - accuracy: 1.0000 - val_loss: 0.3617 - val_accuracy: 0.8571
Epoch 60/2000
49/49 [==============================] - 0s 159us/step - loss: 0.2189 - accuracy: 1.0000 - val_loss: 0.3597 - val_accuracy: 0.8571
Epoch 61/2000
49/49 [==============================] - 0s 139us/step - loss: 0.2130 - accuracy: 1.0000 - val_loss: 0.3576 - val_accuracy: 0.8571
Epoch 62/2000
49/49 [==============================] - 0s 98us/step - loss: 0.2071 - accuracy: 1.0000 - val_loss: 0.3555 - val_accuracy: 0.8571
Epoch 63/2000
49/49 [==============================] - 0s 107us/step - loss: 0.2013 - accuracy: 1.0000 - val_loss: 0.3534 - val_accuracy: 0.8571
Epoch 64/2000
49/49 [==============================] - 0s 151us/step - loss: 0.1956 - accuracy: 1.0000 - val_loss: 0.3513 - val_acc

Epoch 115/2000
49/49 [==============================] - 0s 124us/step - loss: 0.0295 - accuracy: 1.0000 - val_loss: 0.2778 - val_accuracy: 0.8571
Epoch 116/2000
49/49 [==============================] - 0s 100us/step - loss: 0.0284 - accuracy: 1.0000 - val_loss: 0.2779 - val_accuracy: 0.8571
Epoch 117/2000
49/49 [==============================] - 0s 172us/step - loss: 0.0275 - accuracy: 1.0000 - val_loss: 0.2779 - val_accuracy: 0.8571
Epoch 118/2000
49/49 [==============================] - 0s 129us/step - loss: 0.0265 - accuracy: 1.0000 - val_loss: 0.2780 - val_accuracy: 0.8571
Epoch 119/2000
49/49 [==============================] - 0s 153us/step - loss: 0.0256 - accuracy: 1.0000 - val_loss: 0.2782 - val_accuracy: 0.8571
Epoch 120/2000
49/49 [==============================] - 0s 160us/step - loss: 0.0248 - accuracy: 1.0000 - val_loss: 0.2784 - val_accuracy: 0.8571
Epoch 121/2000
49/49 [==============================] - 0s 174us/step - loss: 0.0240 - accuracy: 1.0000 - val_loss: 0.2786 -

Epoch 172/2000
49/49 [==============================] - 0s 116us/step - loss: 0.0068 - accuracy: 1.0000 - val_loss: 0.3148 - val_accuracy: 0.8571
Epoch 173/2000
49/49 [==============================] - 0s 109us/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.3154 - val_accuracy: 0.8571
Epoch 174/2000
49/49 [==============================] - 0s 152us/step - loss: 0.0065 - accuracy: 1.0000 - val_loss: 0.3160 - val_accuracy: 0.8571
Epoch 175/2000
49/49 [==============================] - 0s 145us/step - loss: 0.0064 - accuracy: 1.0000 - val_loss: 0.3166 - val_accuracy: 0.8571
Epoch 176/2000
49/49 [==============================] - 0s 182us/step - loss: 0.0063 - accuracy: 1.0000 - val_loss: 0.3173 - val_accuracy: 0.8571
Epoch 177/2000
49/49 [==============================] - 0s 184us/step - loss: 0.0062 - accuracy: 1.0000 - val_loss: 0.3179 - val_accuracy: 0.8571
Epoch 178/2000
49/49 [==============================] - 0s 176us/step - loss: 0.0061 - accuracy: 1.0000 - val_loss: 0.3185 -

Epoch 229/2000
49/49 [==============================] - 0s 146us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.3423 - val_accuracy: 0.8571
Epoch 230/2000
49/49 [==============================] - 0s 134us/step - loss: 0.0031 - accuracy: 1.0000 - val_loss: 0.3427 - val_accuracy: 0.8571
Epoch 231/2000
49/49 [==============================] - 0s 110us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.3430 - val_accuracy: 0.8571
Epoch 232/2000
49/49 [==============================] - 0s 146us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.3433 - val_accuracy: 0.8571
Epoch 233/2000
49/49 [==============================] - 0s 99us/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.3437 - val_accuracy: 0.8571
Epoch 234/2000
49/49 [==============================] - 0s 132us/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.3440 - val_accuracy: 0.8571
Epoch 235/2000
49/49 [==============================] - 0s 152us/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 0.3443 - 

...

Epoch 1774/2000
49/49 [==============================] - 0s 134us/step - loss: 3.2866e-05 - accuracy: 1.0000 - val_loss: 0.4963 - val_accuracy: 0.8571
Epoch 1775/2000
49/49 [==============================] - 0s 102us/step - loss: 3.2803e-05 - accuracy: 1.0000 - val_loss: 0.4963 - val_accuracy: 0.8571
Epoch 1776/2000
49/49 [==============================] - 0s 122us/step - loss: 3.2759e-05 - accuracy: 1.0000 - val_loss: 0.4964 - val_accuracy: 0.8571
Epoch 1777/2000
49/49 [==============================] - 0s 149us/step - loss: 3.2708e-05 - accuracy: 1.0000 - val_loss: 0.4964 - val_accuracy: 0.8571
Epoch 1778/2000
49/49 [==============================] - 0s 155us/step - loss: 3.2674e-05 - accuracy: 1.0000 - val_loss: 0.4964 - val_accuracy: 0.8571
Epoch 1779/2000
49/49 [==============================] - 0s 152us/step - loss: 3.2650e-05 - accuracy: 1.0000 - val_loss: 0.4965 - val_accuracy: 0.8571
Epoch 1780/2000
49/49 [==============================] - 0s 118us/step - loss: 3.2582e-05 - ac

Epoch 1829/2000
49/49 [==============================] - 0s 111us/step - loss: 3.0655e-05 - accuracy: 1.0000 - val_loss: 0.4980 - val_accuracy: 0.8571
Epoch 1830/2000
49/49 [==============================] - 0s 160us/step - loss: 3.0609e-05 - accuracy: 1.0000 - val_loss: 0.4980 - val_accuracy: 0.8571
Epoch 1831/2000
49/49 [==============================] - 0s 133us/step - loss: 3.0579e-05 - accuracy: 1.0000 - val_loss: 0.4980 - val_accuracy: 0.8571
Epoch 1832/2000
49/49 [==============================] - 0s 174us/step - loss: 3.0545e-05 - accuracy: 1.0000 - val_loss: 0.4981 - val_accuracy: 0.8571
Epoch 1833/2000
49/49 [==============================] - 0s 146us/step - loss: 3.0494e-05 - accuracy: 1.0000 - val_loss: 0.4981 - val_accuracy: 0.8571
Epoch 1834/2000
49/49 [==============================] - 0s 144us/step - loss: 3.0468e-05 - accuracy: 1.0000 - val_loss: 0.4981 - val_accuracy: 0.8571
Epoch 1835/2000
49/49 [==============================] - 0s 169us/step - loss: 3.0433e-05 - ac

Epoch 1884/2000
49/49 [==============================] - 0s 100us/step - loss: 2.8665e-05 - accuracy: 1.0000 - val_loss: 0.4996 - val_accuracy: 0.8571
Epoch 1885/2000
49/49 [==============================] - 0s 145us/step - loss: 2.8636e-05 - accuracy: 1.0000 - val_loss: 0.4997 - val_accuracy: 0.8571
Epoch 1886/2000
49/49 [==============================] - 0s 115us/step - loss: 2.8594e-05 - accuracy: 1.0000 - val_loss: 0.4997 - val_accuracy: 0.8571
Epoch 1887/2000
49/49 [==============================] - 0s 200us/step - loss: 2.8565e-05 - accuracy: 1.0000 - val_loss: 0.4997 - val_accuracy: 0.8571
Epoch 1888/2000
49/49 [==============================] - 0s 160us/step - loss: 2.8519e-05 - accuracy: 1.0000 - val_loss: 0.4997 - val_accuracy: 0.8571
Epoch 1889/2000
49/49 [==============================] - 0s 110us/step - loss: 2.8492e-05 - accuracy: 1.0000 - val_loss: 0.4998 - val_accuracy: 0.8571
Epoch 1890/2000
49/49 [==============================] - 0s 175us/step - loss: 2.8463e-05 - ac

Epoch 1939/2000
49/49 [==============================] - 0s 131us/step - loss: 2.6865e-05 - accuracy: 1.0000 - val_loss: 0.5013 - val_accuracy: 0.8571
Epoch 1940/2000
49/49 [==============================] - 0s 139us/step - loss: 2.6843e-05 - accuracy: 1.0000 - val_loss: 0.5013 - val_accuracy: 0.8571
Epoch 1941/2000
49/49 [==============================] - 0s 163us/step - loss: 2.6821e-05 - accuracy: 1.0000 - val_loss: 0.5014 - val_accuracy: 0.8571
Epoch 1942/2000
49/49 [==============================] - 0s 157us/step - loss: 2.6768e-05 - accuracy: 1.0000 - val_loss: 0.5014 - val_accuracy: 0.8571
Epoch 1943/2000
49/49 [==============================] - 0s 162us/step - loss: 2.6748e-05 - accuracy: 1.0000 - val_loss: 0.5014 - val_accuracy: 0.8571
Epoch 1944/2000
49/49 [==============================] - 0s 142us/step - loss: 2.6709e-05 - accuracy: 1.0000 - val_loss: 0.5015 - val_accuracy: 0.8571
Epoch 1945/2000
49/49 [==============================] - 0s 128us/step - loss: 2.6656e-05 - ac

Epoch 1994/2000
49/49 [==============================] - 0s 109us/step - loss: 2.5201e-05 - accuracy: 1.0000 - val_loss: 0.5030 - val_accuracy: 0.8571
Epoch 1995/2000
49/49 [==============================] - 0s 128us/step - loss: 2.5169e-05 - accuracy: 1.0000 - val_loss: 0.5030 - val_accuracy: 0.8571
Epoch 1996/2000
49/49 [==============================] - 0s 157us/step - loss: 2.5135e-05 - accuracy: 1.0000 - val_loss: 0.5031 - val_accuracy: 0.8571
Epoch 1997/2000
49/49 [==============================] - 0s 146us/step - loss: 2.5099e-05 - accuracy: 1.0000 - val_loss: 0.5031 - val_accuracy: 0.8571
Epoch 1998/2000
49/49 [==============================] - 0s 138us/step - loss: 2.5077e-05 - accuracy: 1.0000 - val_loss: 0.5031 - val_accuracy: 0.8571
Epoch 1999/2000
49/49 [==============================] - 0s 172us/step - loss: 2.5060e-05 - accuracy: 1.0000 - val_loss: 0.5031 - val_accuracy: 0.8571
Epoch 2000/2000
49/49 [==============================] - 0s 144us/step - loss: 2.5040e-05 - ac

In [148]:
acc_test_comb3 = model2_comb2.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

21/21 [==============================] - 0s 107us/step
combination test accuracy: 85.71%


In [149]:
############## Random Forest ##############

In [150]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [151]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 59.04%


In [152]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [153]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 85.71%


In [154]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.48717949 0.46153846 0.66666667 0.64102564 0.43243243]
0.5377685377685377


In [155]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[0.90909091 1.         1.         0.88888889 0.875     ]
0.9345959595959595
